In [17]:
#import the libraries
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from collections import Counter


In [18]:
#read the data
#encoding = "utf-8"
data = pd.read_csv("spam.csv",encoding = "ISO-8859-1")


In [19]:
#printig the data
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [20]:
#droping the unnecessary column
data=data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)


In [21]:
#labeling the data as prediction and text
data=data.rename(columns={'v1':'label','v2':'text'})


In [22]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [23]:
#chacking for null values
data.isnull().sum()


label    0
text     0
dtype: int64

In [24]:
#coverting the Categorical value into numerical value
data.loc[data['Ham/Spam']=='spam','Ham/Spam']=0
data.loc[data['Ham/Spam']=='ham','Ham/Spam']=1

In [25]:
data.head()

,label,text
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [26]:
#seperating the independent and dependent variable
x=data['text']
y=data['Ham/Spam']

In [36]:
#splitting into training and testing data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=0)

In [37]:
#removing the stop words from mail and converting all text to lowercase
vect=TfidfVectorizer(min_df=1,stop_words='english',lowercase=True)

In [38]:
#fit and transform of x_train
#Transform of x_test
x_train_vect=vect.fit_transform(x_train)
x_test_vect=vect.transform(x_test)

In [39]:
#converting the label from str into int data type
y_train=y_train.astype('int')
y_test=y_test.astype('int')

In [31]:
#installing the catboost libraries
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.3 MB/s eta 0:00:00


In [40]:
#building a model with catboost and train it using x_train and y_train variables
from catboost import CatBoostClassifier
model = CatBoostClassifier()
model.fit(x_train_vect,y_train)

Learning rate set to 0.019502
0:	learn: 0.6734753	total: 68.5ms	remaining: 1m 8s
1:	learn: 0.6544376	total: 119ms	remaining: 59.2s
2:	learn: 0.6350619	total: 167ms	remaining: 55.4s
3:	learn: 0.6178361	total: 216ms	remaining: 53.8s
4:	learn: 0.6021651	total: 263ms	remaining: 52.4s
5:	learn: 0.5855916	total: 309ms	remaining: 51.2s
6:	learn: 0.5705613	total: 359ms	remaining: 51s
7:	learn: 0.5547642	total: 410ms	remaining: 50.8s
8:	learn: 0.5410502	total: 457ms	remaining: 50.4s
9:	learn: 0.5262484	total: 519ms	remaining: 51.4s
10:	learn: 0.5129792	total: 569ms	remaining: 51.1s
11:	learn: 0.5005198	total: 624ms	remaining: 51.4s
12:	learn: 0.4882665	total: 671ms	remaining: 51s
13:	learn: 0.4750591	total: 719ms	remaining: 50.7s
14:	learn: 0.4633605	total: 768ms	remaining: 50.4s
15:	learn: 0.4535446	total: 818ms	remaining: 50.3s
16:	learn: 0.4447990	total: 873ms	remaining: 50.5s
17:	learn: 0.4363803	total: 919ms	remaining: 50.2s
18:	learn: 0.4261337	total: 988ms	remaining: 51s
19:	learn: 0.417

In [33]:
#printing the accuracy of model
from sklearn.metrics import accuracy_score
pred=model.predict(x_test_vect)
acc=accuracy_score(y_test,pred)
print('Accuracy: ',acc.round(4)*100,'%')

Accuracy:  97.31 %


In [34]:
#printing the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(x_test_vect)
cm = confusion_matrix(y_test, y_pred)
print(cm)


[[138  28]
 [  2 947]]


In [35]:
#printinh cross validation score
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = model, X = x_train_vect, y = y_train,cv=2)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Learning rate set to 0.014504
0:	learn: 0.6787589	total: 39.9ms	remaining: 39.9s
1:	learn: 0.6632595	total: 74.3ms	remaining: 37.1s
2:	learn: 0.6477071	total: 103ms	remaining: 34.4s
3:	learn: 0.6350827	total: 130ms	remaining: 32.3s
4:	learn: 0.6222673	total: 159ms	remaining: 31.6s
5:	learn: 0.6109436	total: 185ms	remaining: 30.7s
6:	learn: 0.5989816	total: 218ms	remaining: 30.9s
7:	learn: 0.5875681	total: 247ms	remaining: 30.6s
8:	learn: 0.5753840	total: 275ms	remaining: 30.2s
9:	learn: 0.5648388	total: 305ms	remaining: 30.2s
10:	learn: 0.5534163	total: 332ms	remaining: 29.8s
11:	learn: 0.5423445	total: 359ms	remaining: 29.5s
12:	learn: 0.5322457	total: 387ms	remaining: 29.4s
13:	learn: 0.5216538	total: 414ms	remaining: 29.2s
14:	learn: 0.5115348	total: 452ms	remaining: 29.7s
15:	learn: 0.5016558	total: 479ms	remaining: 29.5s
16:	learn: 0.4941044	total: 509ms	remaining: 29.4s
17:	learn: 0.4866478	total: 537ms	remaining: 29.3s
18:	learn: 0.4781139	total: 564ms	remaining: 29.1s
19:	learn